# PointNets


Disclaimer: The visualization files (render_balls_so.cpp, render_balls_so.dll, show3d_balls.py, Show3dBalls.ipynb) and the dataloader (datasets.py) are from https://github.com/fxia22/pointnet.pytorch.

In [5]:
from __future__ import print_function
from show3d_balls import *
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pdb
import torch.nn.functional as F
import numpy as np
import torchvision.datasets as dset
from datasets import PartDataset



### Networks - Benchmark

In [6]:
class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2,1)
        x = torch.bmm(x, trans)
        x = x.transpose(2,1)
        x = F.relu(self.bn1(self.conv1(x)))
        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans

class PointNetCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetCls, self).__init__()
        self.feat = PointNetfeat(global_feat=True)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.drop = nn.Dropout2d(0.7)
        self.relu = nn.ReLU()
    def forward(self, x):
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.drop(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=0), trans

class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feat = PointNetfeat(global_feat=False)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans



### Networks - Experimentation

In [ ]:

class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
#         CHANGE 8
        self.conv1 = torch.nn.Conv1d(3, 128, 1)
#             self.conv1 = torch.nn.Conv1d(3, 64, 1)
#         CHANGE 5
        self.drop1 = nn.Dropout2d(0.1)
    
#         CHANGE 7 + 8
        self.conv2 = torch.nn.Conv1d(128, 256, 1)
#         self.conv2 = torch.nn.Conv1d(64, 128, 1)

#         CHANGE 6
        self.drop2 = nn.Dropout2d(0.1)
    
#         CHANGE 7
        self.conv3 = torch.nn.Conv1d(256, 1024, 1)    
#         self.conv3 = torch.nn.Conv1d(128, 1024, 1)

        self.drop3 = nn.Dropout2d(0.1)

        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        #CHANGE 7+8
#         self.bn1 = nn.BatchNorm1d(64)
#         self.bn2 = nn.BatchNorm1d(128)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
#         CHANGE 5
        x = self.drop1(x)
        x = F.relu(self.bn2(self.conv2(x)))
#         CHANGE 6
        x = self.drop2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.drop3(x)
        x = torch.max(x, 2, keepdim=True)[0]
        
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
#         CHANGE 9
#         self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv1 = torch.nn.Conv1d(3, 128, 1)
#         CHANGE 3
        self.drop1 = nn.Dropout2d(0.1)
#         CHANGE 9
        self.conv2 = torch.nn.Conv1d(128, 256, 1)
#         CHANGE 4
        self.drop2 = nn.Dropout2d(0.1)
    
#         CHANGE 9
        self.conv3 = torch.nn.Conv1d(256, 1024, 1)
        self.drop3 = nn.Dropout2d(0.1)
#             self.conv3 = torch.nn.Conv1d(128, 1024, 1)

#         CHANGE 9
#         self.bn1 = nn.BatchNorm1d(64)
#         self.bn2 = nn.BatchNorm1d(128)
#         self.bn3 = nn.BatchNorm1d(1024)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(1024)
        
        self.global_feat = global_feat
    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2,1)
        x = torch.bmm(x, trans)
        x = x.transpose(2,1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.drop1(x)
        pointfeat = x
#         CHANGE 3
        x = F.relu(self.bn2(self.conv2(x)))
#         CHANGE 4
        x = self.drop2(x)
        x = self.bn3(self.conv3(x))
        x = self.drop3(x)
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans

class PointNetCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetCls, self).__init__()
        self.feat = PointNetfeat(global_feat=True)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
#         CHANGE 2
        self.drop2 = nn.Dropout2d(0.3)
#         CHANGE 1
        self.drop3 = nn.Dropout2d(0.7)
        self.relu = nn.ReLU()
    def forward(self, x):
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
#         CHANGE 2
        x = self.drop2(x)
        x = F.relu(self.bn2(self.fc2(x)))
#         CHANGE 1
        x = self.drop3(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=0), trans

class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feat = PointNetfeat(global_feat=False)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans



### Classification Data Loader

In [7]:


parser = argparse.ArgumentParser()
parser.add_argument('--batchSize', type=int, default=16, help='input batch size')
parser.add_argument('--num_points', type=int, default=2500, help='input batch size')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
# real default 25
# parser.add_argument('--nepoch', type=int, default=3, help='number of epochs to train for')
parser.add_argument('--nepoch', type=int, default=1, help='number of epochs to train for')
parser.add_argument('--outf', type=str, default='cls',  help='output folder')
parser.add_argument('--model', type=str, default = '',  help='model path')

opt, unknown = parser.parse_known_args()
print (opt)

blue = lambda x:'\033[94m' + x + '\033[0m'

opt.manualSeed = random.randint(1, 10000) # fix seed
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = True, npoints = opt.num_points)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize, drop_last=True,
                                          shuffle=True, num_workers=int(opt.workers))

test_dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = True, train = False, npoints = opt.num_points)
testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt.batchSize, drop_last=True,
                                          shuffle=True, num_workers=int(opt.workers))

print(len(dataset), len(test_dataset))
num_classes = len(dataset.classes)
print('classes', num_classes)

try:
    os.makedirs(opt.outf)
except OSError:
    pass


Namespace(batchSize=16, model='', nepoch=1, num_points=2500, outf='cls', workers=4)
Random Seed:  2938
{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}
{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}
15990 1785
classes 16


### Classification Training

In [8]:

# classifier = PointNetCls(k = num_classes, num_points = opt.num_points)
classifier = PointNetCls(k = num_classes)


if opt.model != '':
    classifier.load_state_dict(torch.load(opt.model))


optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
classifier.cuda()

num_batch = len(dataset)/opt.batchSize
train_accuracy = []
train_loss = []
test_accuracy = []
test_loss = []
print_to_file = True
for epoch in range(opt.nepoch):
    for i, data in enumerate(dataloader, 0):
        points, target = data
        points, target = Variable(points), Variable(target[:,0])
        points = points.transpose(2,1)
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, _ = classifier(points)
        loss = F.nll_loss(pred, target)
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' %(epoch, i, num_batch, loss.item(),correct.item() / float(opt.batchSize)))
        if print_to_file == True:
            train_accuracy.append(correct.item() / float(opt.batchSize))
            train_loss.append(loss.item())

#         try i % 10 == 9
        if i % 10 == 9:
            j, data = next(enumerate(testdataloader, 0))
            points, target = data
            points, target = Variable(points), Variable(target[:,0])
            points = points.transpose(2,1)
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _ = classifier(points)
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' %(epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.batchSize)))
            if print_to_file == True:
                test_accuracy.append(correct.item() / float(opt.batchSize))
                test_loss.append(loss.item())

    torch.save(classifier.state_dict(), '%s/cls_model_%d.pth' % (opt.outf, epoch))
    np.savetxt('%s/cls_train_loss_%d.out' % (opt.outf, epoch), train_loss)
    np.savetxt('%s/cls_train_acc_%d.out' % (opt.outf, epoch), train_accuracy)
    np.savetxt('%s/cls_test_loss_%d.out' % (opt.outf, epoch), test_loss)
    np.savetxt('%s/cls_test_acc_%d.out' % (opt.outf, epoch), test_accuracy)



[0: 0/999] train loss: 2.914097 accuracy: 0.000000
[0: 1/999] train loss: 2.783659 accuracy: 0.062500
[0: 2/999] train loss: 2.666628 accuracy: 0.187500
[0: 3/999] train loss: 2.581982 accuracy: 0.125000
[0: 4/999] train loss: 2.436080 accuracy: 0.250000
[0: 5/999] train loss: 2.459324 accuracy: 0.312500
[0: 6/999] train loss: 2.246056 accuracy: 0.625000
[0: 7/999] train loss: 2.184893 accuracy: 0.625000
[0: 8/999] train loss: 2.221623 accuracy: 0.375000
[0: 9/999] train loss: 2.176640 accuracy: 0.500000
[0: 9/999] test loss: 2.772388 accuracy: 0.250000
[0: 10/999] train loss: 2.045044 accuracy: 0.562500
[0: 11/999] train loss: 2.032797 accuracy: 0.687500
[0: 12/999] train loss: 2.273293 accuracy: 0.437500
[0: 13/999] train loss: 2.023523 accuracy: 0.687500
[0: 14/999] train loss: 1.919404 accuracy: 0.625000
[0: 15/999] train loss: 2.001523 accuracy: 0.500000
[0: 16/999] train loss: 1.914336 accuracy: 0.625000
[0: 17/999] train loss: 1.997337 accuracy: 0.625000
[0: 18/999] train loss: 

[0: 143/999] train loss: 1.692303 accuracy: 0.687500
[0: 144/999] train loss: 1.567942 accuracy: 0.750000
[0: 145/999] train loss: 1.442594 accuracy: 0.750000
[0: 146/999] train loss: 1.255275 accuracy: 0.812500
[0: 147/999] train loss: 1.968804 accuracy: 0.562500
[0: 148/999] train loss: 1.196628 accuracy: 0.812500
[0: 149/999] train loss: 1.605058 accuracy: 0.750000
[0: 149/999] test loss: 1.168159 accuracy: 0.812500
[0: 150/999] train loss: 1.448048 accuracy: 0.750000
[0: 151/999] train loss: 1.155988 accuracy: 0.875000
[0: 152/999] train loss: 1.605377 accuracy: 0.562500
[0: 153/999] train loss: 2.219007 accuracy: 0.375000
[0: 154/999] train loss: 1.618337 accuracy: 0.687500
[0: 155/999] train loss: 1.411862 accuracy: 0.750000
[0: 156/999] train loss: 1.532571 accuracy: 0.687500
[0: 157/999] train loss: 1.699034 accuracy: 0.687500
[0: 158/999] train loss: 1.560083 accuracy: 0.687500
[0: 159/999] train loss: 1.565822 accuracy: 0.625000
[0: 159/999] test loss: 1.189824 accuracy: 0.87

[0: 283/999] train loss: 1.619284 accuracy: 0.687500
[0: 284/999] train loss: 1.435076 accuracy: 0.687500
[0: 285/999] train loss: 1.323845 accuracy: 0.687500
[0: 286/999] train loss: 1.139068 accuracy: 0.812500
[0: 287/999] train loss: 1.396970 accuracy: 0.812500
[0: 288/999] train loss: 1.083029 accuracy: 0.812500
[0: 289/999] train loss: 1.304286 accuracy: 0.812500
[0: 289/999] test loss: 1.324608 accuracy: 0.875000
[0: 290/999] train loss: 1.881365 accuracy: 0.625000
[0: 291/999] train loss: 1.772273 accuracy: 0.625000
[0: 292/999] train loss: 1.312330 accuracy: 0.812500
[0: 293/999] train loss: 1.653871 accuracy: 0.562500
[0: 294/999] train loss: 1.136751 accuracy: 0.875000
[0: 295/999] train loss: 1.291690 accuracy: 0.687500
[0: 296/999] train loss: 1.196619 accuracy: 0.812500
[0: 297/999] train loss: 1.544397 accuracy: 0.812500
[0: 298/999] train loss: 1.425739 accuracy: 0.750000
[0: 299/999] train loss: 1.378552 accuracy: 0.687500
[0: 299/999] test loss: 1.299421 accuracy: 0.87

[0: 423/999] train loss: 1.517260 accuracy: 0.625000
[0: 424/999] train loss: 1.208660 accuracy: 0.750000
[0: 425/999] train loss: 1.307612 accuracy: 0.625000
[0: 426/999] train loss: 1.525026 accuracy: 0.750000
[0: 427/999] train loss: 1.157378 accuracy: 0.812500
[0: 428/999] train loss: 1.335378 accuracy: 0.937500
[0: 429/999] train loss: 1.099110 accuracy: 0.875000
[0: 429/999] test loss: 1.239607 accuracy: 1.000000
[0: 430/999] train loss: 1.353616 accuracy: 0.625000
[0: 431/999] train loss: 1.842076 accuracy: 0.625000
[0: 432/999] train loss: 1.229025 accuracy: 0.750000
[0: 433/999] train loss: 1.188440 accuracy: 0.750000
[0: 434/999] train loss: 1.220595 accuracy: 0.812500
[0: 435/999] train loss: 1.322304 accuracy: 0.937500
[0: 436/999] train loss: 1.012988 accuracy: 0.937500
[0: 437/999] train loss: 1.588950 accuracy: 0.625000
[0: 438/999] train loss: 1.343532 accuracy: 0.750000
[0: 439/999] train loss: 2.088938 accuracy: 0.562500
[0: 439/999] test loss: 1.244754 accuracy: 0.87

[0: 563/999] train loss: 1.046629 accuracy: 0.937500
[0: 564/999] train loss: 1.512322 accuracy: 0.500000
[0: 565/999] train loss: 1.255591 accuracy: 0.812500
[0: 566/999] train loss: 1.328157 accuracy: 0.875000
[0: 567/999] train loss: 1.394542 accuracy: 0.812500
[0: 568/999] train loss: 1.442871 accuracy: 0.875000
[0: 569/999] train loss: 1.149307 accuracy: 0.875000
[0: 569/999] test loss: 1.619608 accuracy: 0.750000
[0: 570/999] train loss: 1.020502 accuracy: 0.937500
[0: 571/999] train loss: 1.543327 accuracy: 0.750000
[0: 572/999] train loss: 1.611074 accuracy: 0.625000
[0: 573/999] train loss: 1.599161 accuracy: 0.625000
[0: 574/999] train loss: 1.253530 accuracy: 0.812500
[0: 575/999] train loss: 1.500700 accuracy: 0.750000
[0: 576/999] train loss: 1.587641 accuracy: 0.625000
[0: 577/999] train loss: 1.477039 accuracy: 0.687500
[0: 578/999] train loss: 1.581052 accuracy: 0.562500
[0: 579/999] train loss: 1.669460 accuracy: 0.625000
[0: 579/999] test loss: 1.430786 accuracy: 0.87

[0: 703/999] train loss: 1.294602 accuracy: 0.750000
[0: 704/999] train loss: 1.102539 accuracy: 0.687500
[0: 705/999] train loss: 0.898073 accuracy: 0.937500
[0: 706/999] train loss: 1.327964 accuracy: 0.812500
[0: 707/999] train loss: 1.038390 accuracy: 0.812500
[0: 708/999] train loss: 1.138511 accuracy: 0.875000
[0: 709/999] train loss: 1.230730 accuracy: 0.812500
[0: 709/999] test loss: 1.397503 accuracy: 1.000000
[0: 710/999] train loss: 1.445828 accuracy: 0.750000
[0: 711/999] train loss: 1.213821 accuracy: 0.812500
[0: 712/999] train loss: 1.117676 accuracy: 0.812500
[0: 713/999] train loss: 1.060230 accuracy: 0.875000
[0: 714/999] train loss: 1.401093 accuracy: 0.687500
[0: 715/999] train loss: 1.179552 accuracy: 0.812500
[0: 716/999] train loss: 1.251797 accuracy: 0.750000
[0: 717/999] train loss: 1.535114 accuracy: 0.625000
[0: 718/999] train loss: 1.194952 accuracy: 0.750000
[0: 719/999] train loss: 1.294048 accuracy: 0.875000
[0: 719/999] test loss: 1.424231 accuracy: 0.75

[0: 843/999] train loss: 1.541036 accuracy: 0.625000
[0: 844/999] train loss: 1.032651 accuracy: 0.687500
[0: 845/999] train loss: 1.270627 accuracy: 0.812500
[0: 846/999] train loss: 1.491715 accuracy: 0.562500
[0: 847/999] train loss: 1.348944 accuracy: 0.687500
[0: 848/999] train loss: 1.521184 accuracy: 0.687500
[0: 849/999] train loss: 0.922329 accuracy: 0.812500
[0: 849/999] test loss: 1.375722 accuracy: 0.750000
[0: 850/999] train loss: 1.531682 accuracy: 0.687500
[0: 851/999] train loss: 1.344643 accuracy: 0.687500
[0: 852/999] train loss: 0.967568 accuracy: 0.875000
[0: 853/999] train loss: 1.801008 accuracy: 0.562500
[0: 854/999] train loss: 1.087068 accuracy: 0.750000
[0: 855/999] train loss: 1.012429 accuracy: 0.750000
[0: 856/999] train loss: 1.219481 accuracy: 0.625000
[0: 857/999] train loss: 1.140053 accuracy: 0.875000
[0: 858/999] train loss: 1.182773 accuracy: 0.687500
[0: 859/999] train loss: 1.163329 accuracy: 0.687500
[0: 859/999] test loss: 1.453770 accuracy: 0.81

[0: 983/999] train loss: 1.230865 accuracy: 0.812500
[0: 984/999] train loss: 1.529398 accuracy: 0.625000
[0: 985/999] train loss: 1.466378 accuracy: 0.812500
[0: 986/999] train loss: 2.113492 accuracy: 0.500000
[0: 987/999] train loss: 1.184420 accuracy: 0.875000
[0: 988/999] train loss: 1.274811 accuracy: 0.687500
[0: 989/999] train loss: 1.253401 accuracy: 0.750000
[0: 989/999] test loss: 1.155439 accuracy: 0.812500
[0: 990/999] train loss: 1.402464 accuracy: 0.625000
[0: 991/999] train loss: 1.473159 accuracy: 0.687500
[0: 992/999] train loss: 1.868357 accuracy: 0.375000
[0: 993/999] train loss: 1.298316 accuracy: 0.625000
[0: 994/999] train loss: 1.166165 accuracy: 0.875000
[0: 995/999] train loss: 1.478882 accuracy: 0.687500
[0: 996/999] train loss: 1.286791 accuracy: 0.875000
[0: 997/999] train loss: 1.274364 accuracy: 0.875000
[0: 998/999] train loss: 1.921957 accuracy: 0.625000


### Classification Display

In [ ]:


#showpoints(np.random.randn(2500,3), c1 = np.random.uniform(0,1,size = (2500)))

parser = argparse.ArgumentParser()

parser.add_argument('--model', type=str, default = 'cls/cls_model_2.pth',  help='model path')
parser.add_argument('--num_points', type=int, default=2500, help='input batch size')


opt, unknown = parser.parse_known_args()
print (opt)

test_dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0' , train = False, classification = True,  npoints = opt.num_points)

testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle = True)


# classifier = PointNetCls(k = len(test_dataset.classes), num_points = opt.num_points)
classifier = PointNetCls(k = len(test_dataset.classes))
classifier.cuda()
classifier.load_state_dict(torch.load(opt.model))
classifier.eval()


for i, data in enumerate(testdataloader, 0):
    points, target = data
    points, target = Variable(points), Variable(target[:, 0])
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()
    pred, _ = classifier(points)
    loss = F.nll_loss(pred, target)

    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(target.data).cpu().sum()
    print('i:%d  loss: %f accuracy: %f' %(i, loss.data[0], correct/float(32)))


### Segmentation Data Loader

In [ ]:

parser = argparse.ArgumentParser()
parser.add_argument('--batchSize', type=int, default=32, help='input batch size')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
# real default 25
parser.add_argument('--nepoch', type=int, default=3, help='number of epochs to train for')
parser.add_argument('--outf', type=str, default='seg',  help='output folder')
parser.add_argument('--model', type=str, default = '',  help='model path')

opt, unknown = parser.parse_known_args()
print (opt)

opt.manualSeed = random.randint(1, 10000) # fix seed
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = False, class_choice = ['Chair'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize,
                                          shuffle=True, num_workers=int(opt.workers))

test_dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = False, class_choice = ['Chair'], train = False)
testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt.batchSize,
                                          shuffle=True, num_workers=int(opt.workers))

print(len(dataset), len(test_dataset))
num_classes = dataset.num_seg_classes
print('classes', num_classes)
try:
    os.makedirs(opt.outf)
except OSError:
    pass

blue = lambda x:'\033[94m' + x + '\033[0m'


### Segmentation Training

In [ ]:
classifier = PointNetDenseCls(k = num_classes)

if opt.model != '':
    classifier.load_state_dict(torch.load(opt.model))

optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
classifier.cuda()

num_batch = len(dataset)/opt.batchSize

for epoch in range(opt.nepoch):
    for i, data in enumerate(dataloader, 0):
        points, target = data
        points, target = Variable(points), Variable(target)
        points = points.transpose(2,1) 
        points, target = points.cuda(), target.cuda()   
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, _ = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1,1)[:,0] - 1
        #print(pred.size(), target.size())
        loss = F.nll_loss(pred, target)
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' %(epoch, i, num_batch, loss.item(), correct.item()/float(opt.batchSize * 2500)))
        
        if i % 10 == 0:
            j, data = next(enumerate(testdataloader, 0))
            points, target = data
            points, target = Variable(points), Variable(target)
            points = points.transpose(2,1) 
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _ = classifier(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1,1)[:,0] - 1

            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' %(epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.batchSize * 2500)))
    
    torch.save(classifier.state_dict(), '%s/seg_model_%d.pth' % (opt.outf, epoch))

### Segmentation Display

In [ ]:

#showpoints(np.random.randn(2500,3), c1 = np.random.uniform(0,1,size = (2500)))

parser = argparse.ArgumentParser()

parser.add_argument('--model', type=str, default = 'seg/seg_model_2.pth',  help='model path')
parser.add_argument('--idx', type=int, default = 0,   help='model index')



opt, unknown = parser.parse_known_args()
print (opt)

d = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', class_choice = ['Lamp'], train = False)

idx = opt.idx

print("model %d/%d" %( idx, len(d)))

point, seg = d[idx]
print(point.size(), seg.size())

point_np = point.numpy()



cmap = plt.cm.get_cmap("hsv", 10)
cmap = np.array([cmap(i) for i in range(10)])[:,:3]
gt = cmap[seg.numpy() - 1, :]

classifier = PointNetDenseCls(k = 4)
classifier.load_state_dict(torch.load(opt.model))
classifier.eval()

point = point.transpose(1,0).contiguous()

point = Variable(point.view(1, point.size()[0], point.size()[1]))
pred, _ = classifier(point)
pred_choice = pred.data.max(2)[1]
print(pred_choice)

#print(pred_choice.size())
pred_color = cmap[pred_choice.numpy()[0], :]

#print(pred_color.shape)
showpoints(point_np, gt, pred_color)
